In [44]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn import preprocessing

from sklearn.model_selection import KFold

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

import re

from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from functools import reduce
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy3
from pymystem3 import Mystem


import warnings
warnings.filterwarnings('ignore')

In [45]:
df2 = pd.DataFrame(columns=['text', 'label'])
df1 = pd.DataFrame(columns=['text', 'label'])
df_lem_train = pd.DataFrame(columns=['text', 'label'])
df_lem_test = pd.DataFrame(columns=['text', 'label'])

# достаем обучающую выборку и загружаем в 2 ДатаФрейма: для лемматизации и стемминга
df_train = pd.read_csv('/Users/mac/Downloads/rusentitweet_train.csv')
for i, row in df_train.iterrows():
    if row[1] in ("negative", "positive"):
        df2.loc[len(df2.index)] = row
        df_lem_train.loc[len(df_lem_train.index)] = row
        
df_train = df2

        
# достаем тестову выборку и загружаем в 2 ДатаФрейма: для лемматизации и стемминга
df_test = pd.read_csv('/Users/mac/Downloads/rusentitweet_test.csv')
for i, row in df_test.iterrows():
    if row[1] in ("negative", "positive"):
        df1.loc[len(df1.index)] = row
        df_lem_test.loc[len(df_lem_test.index)] = row
df_test = df1

In [46]:
nltk.download('stopwords')
# Стоп-слова
stop = stopwords.words('russian')

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [47]:
# 2 Функция очистки
def my_clear(df):
    s=[]
    s1 = ""
    regs = [r'https?:\/\/.\S+', r'#\S+', r'@\S+', r'\\r\\n', r'[-.)(,:]']
    glasn = ['о', 'о', 'а', 'я', 'а', 'а', 'и', 'а', 'е', 'у', 'о', 'о', 'а', 'е', 'я', 'о', 'у']
    signs = ['!', '_', '.']

    # приведение всего к нижнему регистру
    for i, row in df.iterrows():
        row[0] = row[0].lower()


    for i, row in df.iterrows():
        # удаление ссыло, хэштегов(и знака решетки и последующего хэштега, потому что слова после хэштэга пишутся слитно)
        # удаление отмеченных пользователей(@name), удаление знаков пунктуации (кроме ! ?)
        for reg in regs:
            row[0] = re.sub(reg, "", row[0])

        # удаление стоп-слов
        row[0] = ' '.join([x for x in row[0].split() if x not in stop])

        # удаление повторящихся подряд смайликов и знаков препинания (оставшихся)
        # удаление подряд идущих гласных (тыыыыы, яяяяяя, аааааа и тд)
        # удаление цифр, так как по сути цифры не имеют никакого значения в данном анализе
        r = row[0]
        for t in range(0, len(row[0])):
            if not(not(r[t].isalpha()) and r[t] == r[t-1]) and not((r[t] in glasn) and r[t] == r[t-1]) and not(r[t].isdigit()):
                s1+=r[t]
        row[0] = s1

        s1 = ""

        r = row[0] + "  "

        #отделение смайлика от слова (чимина🤧 -> чимина 🤧)
        for t in range(0, len(row[0])):
            if not(r[t+1].isalpha()) and r[t]!= " " and not(r[t+1].isdigit()) and (r[t+2].isalpha() or r[t+2]==" "):
                s1+=r[t] + " "
            else:
                s1+=r[t]
        row[0] = s1

        s = []
        s1=""
    return df

In [48]:
# 2 очистка обучающих
df_train = my_clear(df_train)
df_test = my_clear(df_test)

In [49]:
# 3
# стемминг теста
new = SnowballStemmer(language='russian')
for i, row in df_test.iterrows():
    row[0] = ' '.join([new.stem(x) for x in row[0].split()])
    
for i, row in df_train.iterrows():
    row[0] = ' '.join([new.stem(x) for x in row[0].split()])

In [64]:
x_train = []
x_test = []

# создание массива строк
for i, row in df_train.iterrows():
    x_train.append(row[0])

for i, row in df_test.iterrows():
    x_test.append(row[0])
x_train[0:10]

['помойм вкраш чимин 🤧',
 'порядк !',
 'след буд победн закрыва пожела удач !',
 'удивительн гимн удивительн пок ещ сдохл украин',
 'срал биолог',
 'помим алин ещ радост гемоглобин повыс',
 'пиздец че вобщ чувств жизн помота прошл сутк туман',
 'спичк ? зажеч огон тво глаз',
 'эт сам дел очен крут',
 'хоч сказа чтот приятн получа "ид нах ебла противн "']

In [51]:
# 4
vec = TfidfVectorizer(stop_words=None)

# tf-idf test and train
matr = vec.fit_transform(x_train)
matr_test = vec.transform(x_test)


In [52]:
# 5 logreg
model_logreg = LogisticRegression()
model_logreg.fit(matr, df_train['label'])
y_logreg = model_logreg.predict(matr_test)

accuracy = accuracy_score(df_test['label'], y_logreg)
print(accuracy)

0.7611548556430446


In [65]:
# 5 top logreg
importance = model_logreg.coef_
abs_importance = abs(importance)
sorted_index = abs_importance.argsort()
sorted_index = sorted_index[0][::-1]
# топ-20 значимых слов
np.array(vec.get_feature_names_out())[sorted_index[0:20]]

array(['блять', 'любить', 'хороший', 'красивый', 'вау', 'пиздец', 'милый',
       'сука', 'нахуй', 'прекрасный', 'классный', 'умирать', 'нравиться',
       'любовь', 'красиво', 'крутой', 'рад', 'круто', 'обожать', 'вобще'],
      dtype=object)

In [67]:
# 5 random_forest
model_random_forest = RandomForestClassifier()
model_random_forest.fit(matr, df_train['label'])
y_forest = model_random_forest.predict(matr_test)

accuracy = accuracy_score(df_test['label'], y_forest)
print(accuracy)

0.7296587926509186


In [68]:
# 5 top random_forest
importance = model_random_forest.feature_importances_
abs_importance = abs(importance)
sorted_index = abs_importance.argsort()[::-1]
# топ-20 значимых слов
np.array(vec.get_feature_names_out())[sorted_index[0:20]]

array(['любить', 'хороший', 'красивый', 'блять', 'вау', 'это', 'пиздец',
       'нравиться', 'милый', 'любовь', 'классный', 'прекрасный', 'хотеть',
       'крутой', 'красиво', 'умирать', 'рад', 'спасибо', 'сука', 'нахуй'],
      dtype=object)

In [56]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/mac/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [57]:
# clear
df_lem_train = my_clear(df_lem_train)
df_lem_test = my_clear(df_lem_test)

In [58]:
# Mystem - контекстный леммотизатор + pip install pymorphy3-dicts-ru - словари
m = Mystem()

x_test_lem = []
x_train_lem = []

# lemmatize
for i, t in df_lem_train.iterrows():
    lemmas = m.lemmatize(t[0])
    x_train_lem.append("".join(lemmas).strip())
    
for i, t in df_lem_test.iterrows():
    lemmas = m.lemmatize(t[0])
    x_test_lem.append("".join(lemmas).strip())

In [59]:
vec = TfidfVectorizer(stop_words=None)

# tf-idf test and train
matr_train_lem = vec.fit_transform(x_train_lem)
matr_test_lem = vec.transform(x_test_lem)

In [60]:
model_logreg = LogisticRegression()
model_logreg.fit(matr, df_lem_train['label'])
y_logreg = model_logreg.predict(matr_test)

accuracy = accuracy_score(df_lem_test['label'], y_logreg)
print(accuracy)

0.7742782152230971


In [61]:
importance = model_logreg.coef_
abs_importance = abs(importance)
sorted_index = abs_importance.argsort()
sorted_index = sorted_index[0][::-1]
print(sorted_index)
# топ-20 значимых слов
np.array(vec.get_feature_names_out())[sorted_index[0:20]]

[ 530 3091 7271 ...  686 1363 2525]


array(['блять', 'любить', 'хороший', 'красивый', 'вау', 'пиздец', 'милый',
       'сука', 'нахуй', 'прекрасный', 'классный', 'умирать', 'нравиться',
       'любовь', 'красиво', 'крутой', 'рад', 'круто', 'обожать', 'вобще'],
      dtype=object)

In [62]:
model_random_forest = RandomForestClassifier(max_depth=1)
model_random_forest.fit(matr, df_lem_train['label'])
y_forest = model_random_forest.predict(matr_test)

accuracy = accuracy_score(df_lem_test['label'], y_forest)
print(accuracy)

0.5774278215223098


In [63]:
importance = model_random_forest.feature_importances_
abs_importance = abs(importance)
sorted_index = abs_importance.argsort()[::-1]
print(sorted_index)
# топ-20 значимых слов
np.array(vec.get_feature_names_out())[sorted_index[0:20]]

[ 715 3632 6193 ... 5155 5156    0]


array(['вау', 'нахуй', 'сон', 'красивый', 'счастливый', 'приятно',
       'нравиться', 'ебаный', 'хах', 'солнышко', 'кайф', 'крутой',
       'хороший', 'интересно', 'классно', 'вобще', 'обожать', 'душа',
       'весь', 'помнить'], dtype=object)